# Minimal example to create an anki deck with a shared javascript file

## Libraries

In [54]:
import genanki
from pathlib import Path
from gtts import gTTS
import hashlib

## Global variables

In [55]:
MEDIA_DIR = Path("../AnkiWords/data/media")
MEDIA_STROKES_DIR = MEDIA_DIR / "data"
FONT_FILE = MEDIA_DIR / "FZKai.ttf"
HANZI_JS_LIB_FILE = MEDIA_DIR / "_hanzi-writer.min.js"
SHARED_HANZI_JS_NAME = Path("js") / "_shared_hanzi.js"
AUDIO_DIR =  Path("audio")

DECK_ID = 98712345612345
MODEL_ID = 123987456987654

## Generate audio 

In [56]:
def stable_audio_filename(text: str) -> str:
    """Generate deterministic filename"""
    h = hashlib.sha1(text.encode("utf-8")).hexdigest()[:12]
    return f"audio_{h}.mp3"

def generate_audio(text: str) -> str:
    """
    Create audio file for the text if not already present.
    Returns the filename that must be placed inside [sound:...]
    """
    filename = stable_audio_filename(text)
    filepath = AUDIO_DIR / filename

    if not filepath.exists():
        print(f"[AUDIO] generating: {filename}")
        tts = gTTS(text=text, lang="zh")   # Chinese TTS
        tts.save(str(filepath))
    else:
        print(f"[AUDIO] exists: {filename}")

    return filename

## Template

In [57]:
qfmt = f"""
<div id="writer-wrapper" style="display:flex; flex-direction:column; align-items:center; gap:12px;">

  <div id="writer-container"
       style="display:flex; gap:20px; justify-content:center; flex-wrap:nowrap;">
  </div>

  <div id="buttons-row"
     style="
        display:flex;
        flex-direction:row;
        gap:16px;
        justify-content:center;
        align-items:center;
        margin-top:10px;
     ">
  
  <button id="replay-btn"
          style="
            background:#444cf7;
            color:white;
            border:none;
            padding:8px 22px;
            font-size:16px;
            border-radius:20px;
            cursor:pointer;
            transition:0.2s;
          ">
    Rejouer
  </button>

  <button id="audio-btn"
          style="
            background:#f05454;
            color:white;
            border:none;
            padding:8px 22px;
            font-size:16px;
            border-radius:20px;
            cursor:pointer;
          ">
    Audio
  </button>

</div>

  <div id="hanzi-data" style="display:none;">{{char}}</div>

</div>

<!-- Hide default Anki audio button, we'll trigger it via JS -->
<style>
.replay-button.soundLink {{
  display: none !important;
}}
</style>

<script>
console.log("[Template] Rendering card for char={{{{char}}}}");
</script>

<script src="{HANZI_JS_LIB_FILE.name}"></script>
<script src="{SHARED_HANZI_JS_NAME.name}"></script>

<script>
console.log("[Template] After script tags, typeof initHanziWriter =", typeof initHanziWriter);
if (typeof initHanziWriter === "function") {{
    console.log("[Template] Calling initHanziWriter('{{{{char}}}}')");
    initHanziWriter("{{{{char}}}}");
}} else {{
    console.error("[Template] ERROR: initHanziWriter is NOT a function at call time");
}}
</script>

<!-- Hidden audio field so Anki creates its [sound:...] replay button -->
<div id="hidden-audio" style="display:none;">
  {{{{audio_src}}}}
</div>

<!-- AUDIO PLAYER: click the hidden Anki replay button -->
<script>
document.getElementById("audio-btn").onclick = function() {{
    console.log("[AUDIO] custom button clicked");
    const defaultBtn = document.querySelector(".replay-button.soundLink");
    if (defaultBtn) {{
        console.log("[AUDIO] Found .replay-button.soundLink, clicking it");
        defaultBtn.click();
    }} else {{
        console.warn("[AUDIO] No .replay-button.soundLink found on card");
    }}
}};
</script>
"""


In [58]:
json_model = genanki.Model(
    MODEL_ID,
    "Example_JsonHanziSharedJS",
    fields=[{"name": "char"}, {"name": "audio_src"}],  
    templates=[
        {
            "name": "Writer Card",
            "qfmt": qfmt,
            "afmt": "{{FrontSide}}"
        }
    ]
)

## Example deck

In [59]:
## Example deck
deck = genanki.Deck(DECK_ID, "Example Deck - Shared Hanzi JS + Audio")

media_files = [
    str(SHARED_HANZI_JS_NAME),        
    str(HANZI_JS_LIB_FILE),
    str(FONT_FILE),
] + [str(p) for p in MEDIA_STROKES_DIR.glob("*.json")]

for char in ["一", "二", "三", "你好啊"]:
    audio_filename = generate_audio(char)   
    audio_path = AUDIO_DIR / audio_filename
    audio_field = f"[sound:{audio_filename}]"
    deck.add_note(genanki.Note(
        model=json_model,
        fields=[char, audio_field]         
    ))

    media_files.append(str(audio_path))

package = genanki.Package(deck)
package.media_files = media_files

[AUDIO] exists: audio_d274eee8a175.mp3
[AUDIO] exists: audio_1d5639f716b6.mp3
[AUDIO] exists: audio_49ddb069d5b8.mp3
[AUDIO] exists: audio_a8a095bad13e.mp3


In [60]:
package.write_to_file("example_shared_hanzi_js.apkg")
print("Generated example_shared_hanzi_js.apkg")

Generated example_shared_hanzi_js.apkg
